# **<font color = '#3498eb'>OBJETIVES</font>**

· This cosmetics e-commerce has experienced flat growth over the past few months and, in general terms, aims to increase the number of conversions and the AOV (Average Order Value).

· We want to create an RFM (Recency, Frequency, Monetary Value) segmentation and a recommendation system (Identify which products we want to put in front of every client).

· We want to optimize every step of the customer journey (awareness, consideration, purchase, retention, and recommendation).

· Optimize the campaigns.

· Optimize the product catalog.

# **<font color = '#3498eb'>SECTOR, COMPANY</font>**

# **<font color = '#3498eb'>SOME QUESTIONS TO START THE ANALYSIS</font>**

- **Customer Journey**

· What is the typical buying process.

· In each session, how many products are viewed, added to the cart, abandoned, and purchased.

· What has been the trend of these indicators in recent months.

- **Clients**

· How many products does each customer purchase.

· How much does each customer spend.

· Are there "best customers" that we want to identify and treat differently?

· Are customers making repeat purchases in subsequent months?

· What is the average LTV of a customer?

· Can we create personalized campaigns, taking into account the LTV of the customer?

- **Products**

· Which products are most frequently purchased?

· Are there products that are not being purchased?

· Is there a correlation between product price and purchase volume?

· Are there products that are viewed but not purchased?

· Are there products that are removed from the shopping cart?

· Can we make personalized product recommendations for each customer?

# **<font color = '#3498eb'>LIBRARIES</font>**

In [5]:
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
%config IPCompleter.greedy=True
import tabulate # Mejora la legibilidad de los resultados con independencia de la librería usada

import sqlalchemy as sa
from sqlalchemy import inspect

# **<font color = '#3498eb'>DATA LOADING AND UNDERSTANDING</font>**

In [2]:
conn_sqlite = sa.create_engine(r'sqlite:///F:\GDrive_DATA\DS4B\DS_MASTERY\02_BUSINESS_ANALYTICS\07_CASOS\CASO3_OPTIMIZACIÓN_ECOMMERCE\ecommerce.db')

In [4]:
inspect(conn_sqlite).get_table_names()

['2019-Dec', '2019-Nov', '2019-Oct', '2020-Feb', '2020-Jan']

In [3]:
oct19 = pd.read_sql('2019-Oct', conn_sqlite)
nov19 = pd.read_sql('2019-Nov', conn_sqlite)
dec19 = pd.read_sql('2019-Dec', conn_sqlite)
jan19 = pd.read_sql('2020-Jan', conn_sqlite)
feb19 = pd.read_sql('2020-Feb', conn_sqlite)

In [5]:
df_ecom = pd.concat([oct19, nov19, dec19, jan19, feb19], axis=0)

df_ecom

,index,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,68,2019-10-01 00:01:46 UTC,view,5843665,1487580005092295511,None,f.o.x,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,72,2019-10-01 00:01:55 UTC,cart,5868461,1487580013069861041,None,italwax,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,95,2019-10-01 00:02:50 UTC,view,5877456,1487580006300255120,None,jessnail,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,122,2019-10-01 00:03:41 UTC,view,5649270,1487580013749338323,None,concept,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,124,2019-10-01 00:03:44 UTC,view,18082,1487580005411062629,None,cnd,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...,...,...,...
429785,4156660,2020-02-29 23:58:49 UTC,cart,5815662,1487580006317032337,None,None,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429786,4156663,2020-02-29 23:58:57 UTC,view,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429787,4156668,2020-02-29 23:59:05 UTC,cart,5815665,1487580006317032337,None,None,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
429788,4156675,2020-02-29 23:59:28 UTC,view,5817692,1487580010872045658,None,None,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


# **<font color = '#3498eb'>DATA QUALITY</font>**

### Variable analysis

In [ ]:
df_ecom.info()

In [6]:
# I delete the index column

df_ecom.drop(columns='index', inplace=True)

In [3]:
# Convert event_time to datetime

df_ecom.event_time = pd.to_datetime(df_ecom.event_time)

### Null Management

In [12]:
null_count  = df_ecom.isna().sum().sort_values(ascending=False)
null_percentage = (df_ecom.isnull().mean() * 100).sort_values(ascending=False).astype(int)

df_nulls = pd.DataFrame({'Null_Count':null_count, 'Null_Percentage':null_percentage})

df_nulls

,Null_Count,Null_Percentage
category_code,2060411,98
brand,891646,42
user_session,506,0
event_time,0,0
event_type,0,0
product_id,0,0
category_id,0,0
price,0,0
user_id,0,0


In [4]:
# I remove two variables and the null values from the user_session

df_ecom = df_ecom.drop(columns=["category_code", "brand"]).dropna()

df_ecom

,event_time,event_type,product_id,category_id,price,user_id,user_session
0,2019-10-01 00:01:46+00:00,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
1,2019-10-01 00:01:55+00:00,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1
2,2019-10-01 00:02:50+00:00,view,5877456,1487580006300255120,122.22,527418424,86e77869-afbc-4dff-9aa2-6b7dd8c90770
3,2019-10-01 00:03:41+00:00,view,5649270,1487580013749338323,6.19,555448072,b5f72ceb-0730-44de-a932-d16db62390df
4,2019-10-01 00:03:44+00:00,view,18082,1487580005411062629,16.03,552006247,2d8f304b-de45-4e59-8f40-50c603843fe5
...,...,...,...,...,...,...,...
2095071,2020-02-29 23:58:49+00:00,cart,5815662,1487580006317032337,0.92,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095072,2020-02-29 23:58:57+00:00,view,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095073,2020-02-29 23:59:05+00:00,cart,5815665,1487580006317032337,0.59,147995998,5ff96629-3627-493e-a25b-5a871ec78c90
2095074,2020-02-29 23:59:28+00:00,view,5817692,1487580010872045658,0.79,619841242,18af673b-7fb9-4202-a66d-5c855bc0fd2d


### Analysis of Numerical variables 

In [7]:
df_ecom['price'].describe().round(2)

count    2094570.00
mean           8.42
std           19.14
min          -47.62
25%            2.05
50%            4.00
75%            6.86
max          327.78
Name: price, dtype: float64

In [11]:
# Let's see how many prices are zero or negative

df_ecom[df_ecom['price'] <= 0].shape[0]

20544

In [14]:
# Are these rows associated with a specific product?

df_ecom[df_ecom['price'] <= 0]['product_id'].unique()


array([5892052, 5889621, 5889622, ..., 5824841, 5826413, 5832437],
      dtype=int64)

In [16]:
# I remove  all the rows with zero or negative prices

df_ecom = df_ecom[df_ecom['price'] > 0]

### Analysis of Categorical variables

In [17]:
df_ecom.event_type.value_counts()

event_type
view                961558
cart                574547
remove_from_cart    410357
purchase            127564
Name: count, dtype: int64

In [19]:
# How many unique categories are there?

df_ecom.category_id.nunique()

508

In [20]:
# How many unique products are there?

df_ecom.product_id.nunique()

45327

### Index

In [ ]:
# Let's set 'event_time' as the index

df_ecom.set_index('event_time', inplace=True)

In [23]:
df_ecom.head(2)

,event_type,product_id,category_id,price,user_id,user_session
event_time,,,,,,
2019-10-01 00:01:46+00:00,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361
2019-10-01 00:01:55+00:00,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1


# **<font color = '#3498eb'>DATA TRANSFORMATION</font>**

To conduct a more thorough analysis, I'll break down the components of event_time, include local holidays in the EEUU and incorporate days of special commercial interest.

### Components of event_time

In [2]:
def date_components(dataframe):
    date = dataframe.index.date
    year = dataframe.index.year
    month = dataframe.index.month
    day = dataframe.index.day
    hour = dataframe.index.hour
    minute = dataframe.index.minute
    second = dataframe.index.second
    
    return(pd.DataFrame({'date':date, 'year':year,'month':month, 'day':day, 'hour':hour, 'minute':minute, 'second':second}))

In [12]:
df_ecom = pd.concat([df_ecom.reset_index(), date_components(df_ecom)], axis=1).set_index('event_time')

In [29]:
df_ecom.head(2)

,event_type,product_id,category_id,price,user_id,user_session,date,year,month,day,hour,minute,second,date,year,month,day,hour,minute,second
event_time,,,,,,,,,,,,,,,,,,,,
2019-10-01 00:01:46+00:00,view,5843665,1487580005092295511,9.44,462033176,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019,10,1,0,1,46,2019-10-01,2019,10,1,0,1,46
2019-10-01 00:01:55+00:00,cart,5868461,1487580013069861041,3.57,514753614,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019,10,1,0,1,55,2019-10-01,2019,10,1,0,1,55


### Local holidays in EEUU

In [ ]:
%conda install -c conda-forge holidays -y

In [10]:
import holidays

holidays_eeuu = holidays.US(years=2020)

In [ ]:
# I'll add a variable that will indicate whether the record falls on a holiday or not

df_ecom['holiday'] = df_ecom.date.apply(lambda record: 1 if (record in holidays_eeuu) else 0)

In [16]:
df_ecom[df_ecom.holiday == 1].date.value_counts()

date
2019-11-28    22560
2020-02-17    15911
2020-01-20    15328
2019-10-14    13662
2019-11-11    13520
2019-12-25    11046
2020-01-01     7644
Name: count, dtype: int64

### Days of special commercial interest

Since this is a cosmetics e-commerce, I will mark the records for dates that are especially significant for the target audience. My hypothesis is that around these dates we should see an increase in sales.

In [21]:
df_ecom['black_friday'] = 0
df_ecom['2019-11-29','black_friday'] = 1

df_ecom['valentines_day'] = 0
df_ecom.loc['2019-02-14', 'valentines_day'] = 1
df_ecom.loc['2020-02-14', 'valentines_day'] = 1

df_ecom['mothers_day'] = 0
df_ecom.loc['2019-05-12', 'mothers_day'] = 1
df_ecom.loc['2020-05-10', 'mothers_day'] = 1

df_ecom['cyber_monday'] = 0
df_ecom.loc['2019-12-02', 'cyber_monday'] = 1
df_ecom.loc['2020-11-30', 'cyber_monday'] = 1

df_ecom['womens_day'] = 0
df_ecom.loc['2019-03-08', 'womens_day'] = 1
df_ecom.loc['2020-03-08', 'womens_day'] = 1

df_ecom['halloween'] = 0
df_ecom.loc['2019-10-31', 'halloween'] = 1
df_ecom.loc['2020-10-31', 'halloween'] = 1

df_ecom['prime_day'] = 0
df_ecom.loc['2019-07-15', 'prime_day'] = 1
df_ecom.loc['2020-10-13', 'prime_day'] = 1

df_ecom['christmas'] = 0
df_ecom.loc['2019-12-25', 'christmas'] = 1
df_ecom.loc['2020-12-25', 'christmas'] = 1

df_ecom['new_years_eve'] = 0
df_ecom.loc['2019-12-31', 'new_years_eve'] = 1
df_ecom.loc['2020-12-31', 'new_years_eve'] = 1

In [19]:
df_ecom.head(2)

,event_type,product_id,category_id,price,user_id,user_session,date,year,month,day,...,second,holiday,valentines_day,mothers_day,cyber_monday,womens_day,halloween,prime_day,christmas,new_years_eve
event_time,,,,,,,,,,,,,,,,,,,,,
2019-10-01 00:01:46+00:00,view,5843665.0,1.487580e+18,9.44,462033176.0,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019.0,10.0,1.0,...,46.0,0.0,0.0,0,0,0,0,0,0,0
2019-10-01 00:01:55+00:00,cart,5868461.0,1.487580e+18,3.57,514753614.0,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019.0,10.0,1.0,...,55.0,0.0,0.0,0,0,0,0,0,0,0


In [23]:
df_ecom['valentines_day'].value_counts()

valentines_day
0    2061781
1      12246
Name: count, dtype: int64

# **<font color = '#3498eb'>ANALYSIS AND INSIGHTS</font>**

## UNDERSTANDING EVENTS

### How does the customer journey work?

### On average, how many products are viewed, added to the cart, abandoned and purchased in each session?

### Are there differences in events by the hour?

### On average, what is the monthly revenue?

### In the latest month, what is the trend?

### What are the days and hours when most purchases are made?

## UNDERSTANDING CLIENTS

### Introduccion

### How are customers distributed based on their spending?

### How are customers distributed based on the number of purchases?

### On average, how many products does a customer buy in a single purchase?

### Which customers generated the most revenue?

### What is the LTV (Lifetime Value) of customers?

### Which customers are most valuable for targeting in future campaigns?

## UNDERSTANDING PRODUCTS?

### Introduccion

### What are the most purchased products?

### Are there products that haven't been purchased and can be removed from the catalog?

### What is the correlation between the price of a product and its sales volume?

### Are there products that are more frequently removed from the cart?

### What are the most viewed products?

### Are there products that are viewed frequently but not purchased?

### Building a product recommendation system

# **<font color = '#3498eb'>CONCLUSIONS</font>**

# **<font color = 'purple'>I save all the work I did today</font>**

In [10]:
# 29/9/2023 14:42
df_ecom.to_pickle('F:\\GDrive_DATA\\DS4B\\DS_MASTERY\\02_BUSINESS_ANALYTICS\\07_CASOS\\CASO3_OPTIMIZACIÓN_ECOMMERCE\\df_ecomNoindex.pickle')

In [2]:
df_ecom = pd.read_pickle('F:\\GDrive_DATA\\DS4B\\DS_MASTERY\\02_BUSINESS_ANALYTICS\\07_CASOS\\CASO3_OPTIMIZACIÓN_ECOMMERCE\\df_ecomNoindex.pickle')

df_ecom.head(2)

,event_type,product_id,category_id,price,user_id,user_session,date,year,month,day,...,holiday,valentines_day,mothers_day,cyber_monday,womens_day,halloween,prime_day,christmas,new_years_eve,black_friday
event_time,,,,,,,,,,,,,,,,,,,,,
2019-10-01 00:01:46+00:00,view,5843665.0,1.487580e+18,9.44,462033176.0,a18e0999-61a1-4218-8f8f-61ec1d375361,2019-10-01,2019.0,10.0,1.0,...,0.0,0,0,0,0,0,0,0,0,0
2019-10-01 00:01:55+00:00,cart,5868461.0,1.487580e+18,3.57,514753614.0,e2fecb2d-22d0-df2c-c661-15da44b3ccf1,2019-10-01,2019.0,10.0,1.0,...,0.0,0,0,0,0,0,0,0,0,0
